<a href="https://colab.research.google.com/github/mihnin/mihnin/blob/master/ApartmentExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание проекта
В вашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах. Нужно научиться определять рыночную стоимость объектов недвижимости с применением машинного обучения.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Откроем и изучем наш датасет по недвижимости

In [ ]:
!pip install --upgrade pandas
import pandas as pd # Подгружаем нашу библиотеку по работе с табличными данными
from pathlib import Path
# в переменную записываем путь к файлу с которым будем работать
#file = Path.cwd() / 'data' / 'raw' / 'train_data.csv'
file = '/content/drive/MyDrive/datasets/apartment_spb/train_data.csv'
#data = pd.read_csv('c:/Temp/train_data.csv') #Загружаем данные из csv в переменную data
data = pd.read_csv(file)


In [ ]:


data.info() #Анализируме наш датасет на пропущенные значения, кол-во столбцов, строк, типы данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6495 entries, 0 to 6494
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   last_price           6495 non-null   float64
 1   total_area           6495 non-null   float64
 2   rooms                6495 non-null   int64  
 3   ceiling_height       6495 non-null   float64
 4   floors_total         6495 non-null   float64
 5   living_area          6495 non-null   float64
 6   floor                6495 non-null   int64  
 7   is_apartment         6495 non-null   float64
 8   studio               6495 non-null   float64
 9   open_plan            6495 non-null   float64
 10  kitchen_area         6495 non-null   float64
 11  balcony              6495 non-null   float64
 12  airports_nearest     6495 non-null   float64
 13  cityCenters_nearest  6495 non-null   float64
dtypes: float64(12), int64(2)
memory usage: 710.5 KB


In [ ]:
data.describe() # Анализируем значения столбцов на аномальные значения (например отрицательные цены, кол-ва, площади или на слишком максимальные значения), чем меньше аномалий тем более точнее предсказания.

,last_price,total_area,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,airports_nearest,cityCenters_nearest
count,6.495000e+03,6495.000000,6495.000000,6495.000000,6495.000000,6495.000000,6495.000000,6495.000000,6495.0,6495.0,6495.000000,6495.000000,6495.000000,6495.000000
mean,8.050284e+06,65.588209,2.239569,2.780531,10.855427,38.019549,5.922248,0.002309,0.0,0.0,11.057521,0.651578,27996.794303,11460.108699
std,1.175864e+07,39.630351,1.163771,0.687026,6.053443,25.101307,4.635593,0.048005,0.0,0.0,6.575596,1.009999,11581.935337,4776.693612
min,1.219000e+04,17.000000,1.000000,1.000000,1.000000,3.000000,1.000000,0.000000,0.0,0.0,2.000000,0.000000,0.000000,181.000000
25%,4.350000e+06,42.500000,1.000000,2.550000,5.000000,20.000000,3.000000,0.000000,0.0,0.0,7.100000,0.000000,18446.000000,8180.000000
50%,5.650000e+06,56.200000,2.000000,2.700000,9.000000,32.000000,5.000000,0.000000,0.0,0.0,9.500000,0.000000,26402.000000,12255.000000
75%,8.300000e+06,75.150000,3.000000,2.850000,15.500000,45.500000,8.000000,0.000000,0.0,0.0,12.300000,1.000000,36421.000000,14881.000000
max,4.200000e+08,900.000000,16.000000,27.500000,52.000000,409.700000,29.000000,1.000000,0.0,0.0,112.000000,5.000000,54723.000000,29343.000000


In [ ]:
#просматриваем первые и последние строки нашего датасета для общего представления о содержимом.
display(data.head())
display(data.tail())

,last_price,total_area,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,airports_nearest,cityCenters_nearest
0,5400000.0,59.0,2,2.87,4.0,31.5,2,0.0,0.0,0.0,6.6,0.0,20485.0,8180.0
1,13200000.0,109.0,4,3.15,5.0,72.0,2,0.0,0.0,0.0,12.2,0.0,42683.0,8643.0
2,7000000.0,74.5,3,2.58,10.0,49.0,9,0.0,0.0,0.0,10.8,0.0,14078.0,16670.0
3,3200000.0,37.4,1,2.50,9.0,20.0,4,0.0,0.0,0.0,6.2,2.0,17792.0,17699.0
4,6650000.0,64.9,3,2.65,12.0,41.9,11,0.0,0.0,0.0,10.4,0.0,14767.0,10573.0


,last_price,total_area,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,airports_nearest,cityCenters_nearest
6490,5000000.0,40.1,1,2.65,10.0,20.1,6,0.0,0.0,0.0,10.8,1.0,26495.0,14198.0
6491,15350000.0,158.5,7,2.95,7.0,57.4,7,0.0,0.0,0.0,12.4,1.0,23546.0,3088.0
6492,6850000.0,62.1,2,2.75,20.0,31.0,6,0.0,0.0,0.0,13.9,2.0,34506.0,14375.0
6493,5650000.0,66.0,3,2.50,12.0,42.2,6,0.0,0.0,0.0,10.5,0.0,12892.0,11225.0
6494,4500000.0,48.0,1,2.70,9.0,20.0,9,0.0,0.0,0.0,13.0,2.0,33641.0,14027.0


### Итого: 
загрузили датасет, проверили датасет на пропущенные значения, проанализировали данные и статистические параметры т.е. минимальное и максимальное значение - аномалий не выявили, следовательно, проводить операции по актуализации датасета как в части устранения пропущенных значений, изменения типов и аномалий ненужно. Из анализа датасета мы видим, что планируется обучить модель и предсказать целевой признак - last_price. Целевой признак - колличественный, следовательно решается задача -  регрессии.

Для информации: Целевой признак бывает категориальным и количественным. Если он категориальный, то решается задача классификации; если количественный — регрессии.

Можно провести корреляционный анализ факторов, как мы видим высокая положительная корреляция между кол-вом комнат и общей площадью. Площадью кухни и общей площадью и т.п.
Корреляция - это лишь описание того, есть ли линейная зависимость между признаками. Ее отсутствие говорит, что для линейных моделей (линейная регрессия, логистическая регрессия) этот признак бесполезен. При этом в реальности зависимость может быть очень сильной, просто нелинейной. Сложные алгоритмы типа случайного леса могут эту нелинейную зависимость использовать и выдавать более точный результат. Поэтому на основе только лишь нулевого коэффициента корреляции нельзя отбрасывать признаки.
Корреляция бывает отрицательной так и положительной и принимает значения от -1 до 1.

In [ ]:
data.corr()

,last_price,total_area,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,airports_nearest,cityCenters_nearest
last_price,1.000000,0.690985,0.401289,0.143590,-0.043105,0.586246,0.016424,0.001725,NaN,NaN,0.551718,-0.020138,-0.019294,-0.282104
total_area,0.690985,1.000000,0.786492,0.200534,-0.097935,0.944422,-0.014566,-0.007330,NaN,NaN,0.636764,-0.017995,-0.030509,-0.359766
rooms,0.401289,0.786492,1.000000,0.132938,-0.252181,0.858931,-0.149514,-0.023687,NaN,NaN,0.315568,-0.044524,-0.056490,-0.302575
ceiling_height,0.143590,0.200534,0.132938,1.000000,-0.071115,0.193102,-0.025375,0.111506,NaN,NaN,0.168639,-0.017185,-0.049065,-0.245258
floors_total,-0.043105,-0.097935,-0.252181,-0.071115,1.000000,-0.194121,0.641985,-0.012098,NaN,NaN,0.114246,0.253471,0.129451,0.375241
living_area,0.586246,0.944422,0.858931,0.193102,-0.194121,1.000000,-0.088492,-0.010745,NaN,NaN,0.468889,-0.045201,-0.052952,-0.365492
floor,0.016424,-0.014566,-0.149514,-0.025375,0.641985,-0.088492,1.000000,0.003575,NaN,NaN,0.119549,0.238255,0.093441,0.224057
is_apartment,0.001725,-0.007330,-0.023687,0.111506,-0.012098,-0.010745,0.003575,1.000000,NaN,NaN,0.002360,0.042007,0.012462,0.023974
studio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
open_plan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Необходимо наш датасет разбить на 3 части (20-25%(тестовая) + 75-80%(учебная (75-80%), валидационная(20-25%)))

In [ ]:
from sklearn.model_selection import train_test_split #подключаем библиотеку, которая умеет нарезать датасет на учебный, валидационный датасеты

data_test = data.loc[0: 1200, :] # 20%  датасета заранее отщипываем и записываем в тестовую выборку, тестовую выборку. 
# Тестовую выборку применяем только когда уже обучили модель и получили желаемые результаты гиперпараметров (т.е. другими словами это как дипломный проект наших обученных моделей)

data_train_valid = data.loc[1201: 6494, :] # отсавшиеся 80% записываем в переменную из которой будум выделять тренировочную и валидационную выборку
# избавляемся от целевого признака итоговая стоимость квартиры
features = data_train_valid.drop(['last_price'], axis=1)  # извлекаем целевой признак в целевую переменную
target = data_train_valid['last_price']  # создаем перменную target из значений целевого признаков

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target , test_size=0.25, random_state=12345) # начинаем разбивать наши 80% данных на учебные и валидационные переменные в пропорции 75 и 25%, где 25% относим на валидационную выборку.

In [ ]:
# Анализируем сколько строк и столбцов нашего первоначального датасета приходится на тестовую, учебную и валидационную выборку
print('кол-во строк и столбцов в тестовом датасете',data_test.shape)
print('кол-во строк и столбцов в обучающейся выборке',features_train.shape, target_train.shape)
print('кол-во строк и столбцов в валидационной выборке',features_valid.shape, target_valid.shape)

кол-во строк и столбцов в тестовом датасете (1201, 14)
кол-во строк и столбцов в обучающейся выборке (3970, 13) (3970,)
кол-во строк и столбцов в валидационной выборке (1324, 13) (1324,)


### Итого:
Разбили наш датасет на тестовый и (обучающий + валидационный) в пропорции 20% на 80%. где из 80% выделили 25% на валидационный  датасет.
Определили целевой признак в переменной target и обучающие признаки в переменной features

## Исследуйте модели

### Рассмотрим одну популярную модель — решающее дерево. Оно может описывать процесс принятия решения почти в любой ситуации.

In [ ]:
# Для работы с моделью решающее дерево как обычно подгружаем необходимые библиотеки
from sklearn.tree import DecisionTreeRegressor # библиотека отвечающая за создание модели решающего дерева
# библиотека отвечающая за проверку качества модели в нашем случае это параметр MAE (средняя абсолютная ошибка)
from sklearn.metrics import mean_absolute_error


Каким должен быть гиперпараметр maх_depth, чтобы улучшить нашу модель? Неизвестно. Поэтому в цикле перебираем разные значения и сравниваем качество моделей в разных вариантах. Сейчас мы будем проверять автоматически, не дожидаясь проверки на тестовой выборке.

In [ ]:
%%time  
# для информации поставим таймер и замерим как долго обучается модель на учебном датасете
best_model = None
best_result = 10000
best_depth = 0
best_error = 0
for depth in range(1, 10):
    #создаем модель, с 2-мя глобальными гиперпараметрами, где с помощью цикла нйдем значение глубины решающего дерева переменной depth
    model_tree = DecisionTreeRegressor(random_state=12345, max_depth=depth)
    # обучаем нашу модель на учебной выборке
    model_tree.fit(features_train, target_train)
    # Проверяме нашу модель, т.е. подставляя занчения features_valid
    predictions_valid_tree = model_tree.predict(features_valid)# < найдите предсказания на валидационной выборке >
    # проверяем качество нашей модели т.е. сраниваем предсказания модели с результатами target_valid и находим MAE (среднюю абсолютную ошибку)
    result_tree_1 = mean_absolute_error(target_valid, predictions_valid_tree)
    best_model = model_tree
    best_depth = depth
    best_error = result_tree_1
    print(best_model,"Глубина дерева:", best_depth, "погрешность в руб", best_error.round(decimals=1))


DecisionTreeRegressor(max_depth=1, random_state=12345) Глубина дерева: 1 погрешность в руб 4282452.2
DecisionTreeRegressor(max_depth=2, random_state=12345) Глубина дерева: 2 погрешность в руб 3334364.7
DecisionTreeRegressor(max_depth=3, random_state=12345) Глубина дерева: 3 погрешность в руб 2720106.4
DecisionTreeRegressor(max_depth=4, random_state=12345) Глубина дерева: 4 погрешность в руб 2538132.4
DecisionTreeRegressor(max_depth=5, random_state=12345) Глубина дерева: 5 погрешность в руб 2532280.6
DecisionTreeRegressor(max_depth=6, random_state=12345) Глубина дерева: 6 погрешность в руб 2373743.7
DecisionTreeRegressor(max_depth=7, random_state=12345) Глубина дерева: 7 погрешность в руб 2287309.7
DecisionTreeRegressor(max_depth=8, random_state=12345) Глубина дерева: 8 погрешность в руб 2358411.0
DecisionTreeRegressor(max_depth=9, random_state=12345) Глубина дерева: 9 погрешность в руб 2345640.6
Wall time: 125 ms


#### Наилучший показатель качества модели "решающее дерево" достигается при значении  гиперпараметров max_depth=7, ошибка предсказания модели в среднем составляет +- 2,3 млн.руб

### Попробуем новый алгоритм классификации — случайный лес (англ. random forest). Алгоритм обучает большое количество независимых друг от друга деревьев, а потом принимает решение на основе голосования. Случайный лес помогает улучшить результат предсказания и избежать переобучения.

In [ ]:
# Для работы с моделью случайный лес как обычно подгружаем необходимые библиотеки
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error


In [ ]:
%%time
best_model = None
best_est = 0
best_depth = 0
best_error = 0
# Обучаем модель случайного леса для задачи регрессии:
for est in range(10, 51, 10):  # с количеством деревьев: от 10 до 50 с шагом 10,
    for depth in range(1, 11): # с максимальной глубиной от 1 до 10.
        # обучите модель с заданным количеством деревьев
        model_forest = RandomForestRegressor(
            random_state=12345, n_estimators=est, max_depth=depth)
        # обучаем модель на тренировочной выборке
        model_forest.fit(features_train, target_train)
        # получаем предсказания модели на валидационной выборке
        predictions_valid_forest = model_forest.predict(features_valid)
        result_forest_1 = mean_absolute_error(
            target_valid, predictions_valid_forest)  # находим MAE (средняя абсолютная ошибка)
        if result_forest_1 < 2000000:  # MAE (средняя абсолютная ошибка) не более 2 млн рублей
            best_model = model_forest  # сохраните наилучшую модель
            best_est = est
            best_depth = depth
            best_error = result_forest_1
print(best_model, "Количество деревьев:", best_est, "Максимальная глубина:",
      depth, "погрешность в руб", best_error.round(decimals=1))


RandomForestRegressor(max_depth=10, n_estimators=50, random_state=12345) Количество деревьев: 50 Максимальная глубина: 10 погрешность в руб 1870648.3
Wall time: 9.42 s


#### Наилучший показатель качества модели "случайный лес" достигается при значении  гиперпараметров n_estimators=50, глубины max_depth=10 и параметра random_state=12345
Чтобы управлять количеством деревьев в лесу, гиперпараметр n_estimators* (от англ. number of estimators, «количество оценщиков»). 
Чем больше деревьев, тем дольше модель будет учиться, но результат станет лучше (и наоборот).

### Если сделать гиперпараметр n_estimators больше, модель начнёт разрастаться и медленно обучаться. Это плохо. Мало деревьев и результаты не лучше. Тоже плохо. Сколько можно быть зависимым от деревьев? Попробуем ещё один алгоритм — логистическую регрессию.

In [ ]:
# Для работы с моделью линейная регрессия как обычно подгружаем необходимые библиотеки
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [ ]:
%%time
# создаем модель линейной регрессии
model_reg = LinearRegression()
model_reg.fit(features_train,target_train ) # Обучаем модель линейно регрессии 
# получите предсказания модели на валидационной выборке
predictions_valid_reg = model_reg.predict(features_valid)
# находим MAE (средняя абсолютная ошибка)
result_reg1 = mean_absolute_error(target_valid, predictions_valid_reg)

print(model_reg, "погрешность в руб", result_reg1.round(decimals=1))


LinearRegression() погрешность в руб 2781574.3
Wall time: 58.6 ms


### Итого: 
Cредняя абсолютная ошибка (англ. Mean Absolute Error, MAE). Это один из важных критериев для бизнеса: чем меньше значение, тем лучше.
Самое высокое качество у модели случайный лес.
На втором месте модель — решающее дерево.
Самое низкое качество  у модели линейной  регрессии. 

https://neerc.ifmo.ru/wiki/index.php?title=Оценка_качества_в_задачах_классификации_и_регрессии 

## Проверьте модель на тестовой выборке

In [ ]:
# Когда наши модели обучены и мы уже знаем какие гипер параметры необходимо использовать, чтобы добиться максимальной результативности. Пришла пора провести итоговый экзамен наших моделей на тестовой выборке

# извлекаем целевой признак из тестовой выборки
test_features = data_test.drop(['last_price'], axis=1)
test_target = data_test['last_price']   # отражаем целевой признак в целевой переменной


Обьединяем выборку train + valid т.е. обьем обучения увеличиваем на величину валидационный выборки,  
которую ранее использовали для проверки, т.к. сейчас все будем проверять на тестовой выборке + подставим уже известные нам гипер параметры и проверим полученный результат

In [ ]:
model_tree_v2 = DecisionTreeRegressor(random_state=12345, max_depth=7) # создаем модель решающее дерево с уже известными нам параметрами
model_tree_v2.fit(features, target) # обучаем нашу модель на данных учебной и валидационной выборки
test_predictions_tree_v2 = model_tree_v2.predict(test_features) # подставляем в обученную нами модель тестовые данные для предсказания цены квартир
test_result_tree_v2 = mean_absolute_error(test_target, test_predictions_tree_v2)  # проверяем качество модели в нашем случае это параметр MAE (средняя абсолютная ошибка)

print("Тестовая выборка: решающее дерево обученная на train+valid",
      test_result_tree_v2.round(decimals=1))  # выводим полученный результат

model_forest_v2 = RandomForestRegressor(
    random_state=12345, n_estimators=50, max_depth=10)  # создаем модель случайный лес с уже известными нам параметрами
# обучаем нашу модель на данных учебной и валидационной выборки
model_forest_v2.fit(features, target)

# подставляем в обученную нами модель тестовые данные для предсказания цены квартир
test_predictions_forest_v2 = model_forest_v2.predict(test_features)
test_result_forest_v2 = mean_absolute_error(
    test_target, test_predictions_forest_v2)  # проверяем качество модели в нашем случае это параметр MAE (средняя абсолютная ошибка)

print("Тестовая выборка: случайный лес обученная на train+valid",
      test_result_forest_v2.round(decimals=1))  # выводим полученный результат

# создаем модель линейная регрессия
model_reg_v2 = LinearRegression()
model_reg_v2.fit(features,target ) # обучаем нашу модель
# подставляем в обученную нами модель тестовые данные для предсказания цены квартир
test_predictions_reg_v2 = model_reg_v2.predict(test_features)
test_accuracy_reg_v2 = mean_absolute_error(
    test_target, test_predictions_reg_v2)  # проверяем качество модели в нашем случае это параметр MAE (средняя абсолютная ошибка)

print("Тестовая выборка: линейная регрессия обученная на train+valid",
      test_accuracy_reg_v2.round(decimals=1))  # выводим полученный результат


Тестовая выборка: решающее дерево обученная на train+valid 2442096.8
Тестовая выборка: случайный лес обученная на train+valid 1990532.6
Тестовая выборка: линейная регрессия обученная на train+valid 2948783.8


Наилучший результат показала модель - случайный лес

### Итого:
Лучше себя показала как на тестовой так и на валидационной выборке - модель случайный лес

## (бонус) Проверьте модели на адекватность

In [ ]:
# Создаем тестовые данные для предсказания стоимости квартиры
new_features3 = new_features = pd.DataFrame(
    [[59.0, 2, 2.87, 4.0, 31.5, 2, 0.0, 0.0, 0.0, 6.6, 0.0, 20485.0, 8180.0],
     [109.0, 4, 3.15, 5.0, 72.0, 2, 0.0, 0.0, 0.0, 12.2, 0.0, 42683.0, 8643.0]],
    columns=features.columns)



result_my_model = model_forest_v2.predict(new_features3)
result_my_model2 = model_tree_v2.predict(new_features3)
result_my_model3 = model_reg_v2.predict(new_features3)
print('Результаты модели решающее дерево', result_my_model.round(decimals=1))
print('Результаты модели случайный лес', result_my_model2.round(decimals=1))
print('Результаты модели линейная регрессия',
      result_my_model3.round(decimals=1))


Результаты модели решающее дерево [ 6418563.8 12694286.9]
Результаты модели случайный лес [ 6513437.2 10073919. ]
Результаты модели линейная регрессия [ 7826357.4 13927168.5]


In [ ]:
data.head(2) #проверяме наши результаты, сколько на самом деле стоят квартиры 


,last_price,total_area,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,airports_nearest,cityCenters_nearest
0,5400000.0,59.0,2,2.87,4.0,31.5,2,0.0,0.0,0.0,6.6,0.0,20485.0,8180.0
1,13200000.0,109.0,4,3.15,5.0,72.0,2,0.0,0.0,0.0,12.2,0.0,42683.0,8643.0


### Итого: 
Фокусник был пьян, фокус не удался)))
Ну а если серьезно, на точное качество предсказания влияет множество факторов (1: обьем обучающего датасета 2: Качество данных 3: Модели предсказания (мы рассмотрели только 3, не прибегая еще к нейроным сетям и т.п. )